# ✅ Environment Validation - GenAI

**Module :** 00-GenAI-Environment  
**Niveau :** 🟢 Débutant  
**Technologies :** Validation complète, Tests d'intégration  
**Durée estimée :** 20 minutes  

## 🎯 Objectifs d'Apprentissage

- [ ] Valider l'environnement GenAI complet
- [ ] Tester toutes les connexions API
- [ ] Vérifier la compatibilité MCP
- [ ] Générer un rapport de santé final

## 📚 Prérequis

- Environment Setup (notebook 00-1) complété
- Docker Services (notebook 00-2) complété
- API Configuration (notebook 00-3) complété


In [ ]:
# Paramètres Papermill - JAMAIS modifier ce commentaire

# Configuration validation
notebook_mode = "interactive"  # "interactive" ou "batch"
skip_widgets = False           # True pour mode batch MCP
debug_level = "INFO"           

# Tests à effectuer
test_apis = True               # Test connexions API
test_docker = True             # Test services Docker
test_notebooks = True          # Test exécution notebooks
test_mcp = True                # Test compatibilité MCP

# Configuration rapport
generate_report = True         # Générer rapport final
save_results = True            # Sauvegarder résultats
export_config = True           # Exporter configuration validée

In [ ]:
# Setup environnement et imports
import os
import sys
import json
import requests
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any
import logging

# Import helpers GenAI
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.genai_helpers import setup_genai_logging, load_genai_config, validate_apis
        print("✅ Helpers GenAI importés")
    except ImportError:
        print("⚠️  Helpers GenAI non disponibles")

# Configuration logging
logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('env_validation')

print(f"✅ Environment Validation GenAI")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔍 Tests: API={test_apis}, Docker={test_docker}, Notebooks={test_notebooks}, MCP={test_mcp}")

In [ ]:
# Validation complète de l'environnement
print("\n" + "="*60)
print("🔍 VALIDATION COMPLÈTE ENVIRONNEMENT GENAI")
print("="*60)

validation_results = {
    'timestamp': datetime.now().isoformat(),
    'tests': {},
    'overall_status': 'unknown',
    'recommendations': []
}

# 1. Test APIs
if test_apis:
    print("\n🌐 TEST CONNEXIONS API")
    api_results = {'status': 'passed', 'details': {}}
    
    # OpenRouter
    try:
        openrouter_key = os.getenv('OPENROUTER_API_KEY')
        if openrouter_key:
            headers = {'Authorization': f'Bearer {openrouter_key}'}
            response = requests.get('https://openrouter.ai/api/v1/models', 
                                  headers=headers, timeout=10)
            if response.status_code == 200:
                models_count = len(response.json().get('data', []))
                print(f"  ✅ OpenRouter: {models_count} modèles disponibles")
                api_results['details']['openrouter'] = {'status': 'ok', 'models': models_count}
            else:
                print(f"  ❌ OpenRouter: HTTP {response.status_code}")
                api_results['details']['openrouter'] = {'status': 'error', 'code': response.status_code}
                api_results['status'] = 'failed'
        else:
            print("  ⚠️  OpenRouter: Clé API manquante")
            api_results['details']['openrouter'] = {'status': 'missing_key'}
    except Exception as e:
        print(f"  ❌ OpenRouter: {str(e)[:50]}...")
        api_results['details']['openrouter'] = {'status': 'error', 'message': str(e)}
        api_results['status'] = 'failed'
    
    # OpenAI (optionnel)
    openai_key = os.getenv('OPENAI_API_KEY')
    if openai_key:
        try:
            headers = {'Authorization': f'Bearer {openai_key}'}
            response = requests.get('https://api.openai.com/v1/models',
                                  headers=headers, timeout=10)
            if response.status_code == 200:
                print("  ✅ OpenAI: Accessible")
                api_results['details']['openai'] = {'status': 'ok'}
            else:
                print(f"  ⚠️  OpenAI: HTTP {response.status_code}")
                api_results['details']['openai'] = {'status': 'warning'}
        except Exception as e:
            print(f"  ⚠️  OpenAI: {str(e)[:50]}...")
            api_results['details']['openai'] = {'status': 'warning'}
    else:
        print("  ℹ️  OpenAI: Non configuré (optionnel)")
        api_results['details']['openai'] = {'status': 'not_configured'}
    
    validation_results['tests']['apis'] = api_results

# 2. Test Docker
if test_docker:
    print("\n🐳 TEST SERVICES DOCKER")
    docker_results = {'status': 'passed', 'details': {}}
    
    try:
        import subprocess
        result = subprocess.run(['docker', '--version'], 
                              capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            print(f"  ✅ Docker: {result.stdout.strip()}")
            docker_results['details']['docker_cli'] = {'status': 'ok'}
            
            # Test services communs
            import socket
            services_ports = {'ComfyUI': 8188, 'Ollama': 11434, 'Jupyter': 8888}
            
            for service, port in services_ports.items():
                try:
                    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                    sock.settimeout(1)
                    result = sock.connect_ex(('localhost', port))
                    sock.close()
                    
                    if result == 0:
                        print(f"  ✅ {service} (:{port}): Accessible")
                        docker_results['details'][service.lower()] = {'status': 'running', 'port': port}
                    else:
                        print(f"  ⚠️  {service} (:{port}): Non accessible")
                        docker_results['details'][service.lower()] = {'status': 'stopped', 'port': port}
                except Exception:
                    print(f"  ❓ {service} (:{port}): Test échoué")
                    docker_results['details'][service.lower()] = {'status': 'unknown', 'port': port}
        else:
            print("  ❌ Docker: CLI non accessible")
            docker_results['status'] = 'failed'
            docker_results['details']['docker_cli'] = {'status': 'error'}
    except Exception as e:
        print(f"  ❌ Docker: {str(e)[:50]}...")
        docker_results['status'] = 'failed'
        docker_results['details']['error'] = str(e)
    
    validation_results['tests']['docker'] = docker_results

print(f"\n📊 Tests complétés à {datetime.now().strftime('%H:%M:%S')}")

In [ ]:
# Test compatibilité MCP et notebooks
if test_mcp or test_notebooks:
    print("\n🔧 TEST COMPATIBILITÉ MCP ET NOTEBOOKS")

# Test MCP
if test_mcp:
    mcp_results = {'status': 'passed', 'details': {}}
    
    # Test variables d'environnement MCP
    mcp_vars = ['GENAI_ROOT', 'GENAI_OUTPUT_DIR']
    for var in mcp_vars:
        value = os.getenv(var)
        if value:
            print(f"  ✅ {var}: {value}")
            mcp_results['details'][var.lower()] = {'status': 'ok', 'value': value}
        else:
            print(f"  ⚠️  {var}: Non défini")
            mcp_results['details'][var.lower()] = {'status': 'missing'}
    
    # Test structure de fichiers
    expected_paths = [
        GENAI_ROOT / 'shared' / 'helpers',
        GENAI_ROOT / '.env.template',
        GENAI_ROOT / 'requirements.txt'
    ]
    
    for path in expected_paths:
        if path.exists():
            print(f"  ✅ Structure: {path.name}")
            mcp_results['details'][f"path_{path.name}"] = {'status': 'ok'}
        else:
            print(f"  ❌ Structure: {path.name} manquant")
            mcp_results['details'][f"path_{path.name}"] = {'status': 'missing'}
            mcp_results['status'] = 'failed'
    
    validation_results['tests']['mcp'] = mcp_results

# Test notebooks
if test_notebooks:
    notebook_results = {'status': 'passed', 'details': {}}
    
    # Compter notebooks par module
    modules = ['00-GenAI-Environment', '01-Images-Foundation', 
               '02-Images-Advanced', '03-Images-Orchestration', '04-Images-Applications']
    
    total_notebooks = 0
    for module in modules:
        module_path = GENAI_ROOT / module
        if module_path.exists():
            notebooks = list(module_path.glob('*.ipynb'))
            count = len(notebooks)
            total_notebooks += count
            print(f"  ✅ {module}: {count} notebooks")
            notebook_results['details'][module] = {'status': 'ok', 'count': count}
        else:
            print(f"  ❌ {module}: Module manquant")
            notebook_results['details'][module] = {'status': 'missing', 'count': 0}
            notebook_results['status'] = 'failed'
    
    print(f"  📊 Total: {total_notebooks} notebooks trouvés")
    notebook_results['total_count'] = total_notebooks
    
    validation_results['tests']['notebooks'] = notebook_results

In [ ]:
# Génération du rapport final
print("\n" + "="*60)
print("📋 RAPPORT FINAL - VALIDATION ENVIRONNEMENT")
print("="*60)

# Calculer le statut global
total_tests = len(validation_results['tests'])
passed_tests = sum(1 for test in validation_results['tests'].values() 
                  if test['status'] == 'passed')
failed_tests = total_tests - passed_tests

if failed_tests == 0:
    validation_results['overall_status'] = 'success'
    status_icon = "✅"
    status_color = "Green"
elif failed_tests <= total_tests // 2:
    validation_results['overall_status'] = 'partial'
    status_icon = "⚠️"
    status_color = "Yellow"
else:
    validation_results['overall_status'] = 'failed'
    status_icon = "❌"
    status_color = "Red"

print(f"\n{status_icon} STATUT GLOBAL: {validation_results['overall_status'].upper()}")
print(f"📊 Tests réussis: {passed_tests}/{total_tests}")
print(f"⚠️  Tests échoués: {failed_tests}/{total_tests}")

# Recommandations
recommendations = []

if validation_results['tests'].get('apis', {}).get('status') == 'failed':
    recommendations.append("Vérifier les clés API dans le fichier .env")

if validation_results['tests'].get('docker', {}).get('status') == 'failed':
    recommendations.append("Installer et démarrer Docker Desktop")
    recommendations.append("Lancer les services: docker compose -f docker-compose.genai.yml up -d")

if validation_results['tests'].get('notebooks', {}).get('total_count', 0) < 16:
    expected = 16
    actual = validation_results['tests'].get('notebooks', {}).get('total_count', 0)
    recommendations.append(f"Compléter les templates notebooks ({actual}/{expected})")

if recommendations:
    print("\n💡 RECOMMANDATIONS:")
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec}")
        
    validation_results['recommendations'] = recommendations

# Prochaines étapes
print("\n🚀 PROCHAINES ÉTAPES:")
if validation_results['overall_status'] == 'success':
    next_steps = [
        "Environnement validé - Prêt pour la génération d'images",
        "Commencer avec le notebook 01-1-OpenAI-DALL-E-3",
        "Explorer les notebooks avancés selon vos besoins"
    ]
else:
    next_steps = [
        "Corriger les problèmes identifiés",
        "Relancer la validation après corrections",
        "Consulter la documentation de dépannage si nécessaire"
    ]

for i, step in enumerate(next_steps, 1):
    print(f"{i}. {step}")

print(f"\n⏰ Validation terminée: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Sauvegarde des résultats
if save_results:
    try:
        output_dir = Path(os.getenv('GENAI_OUTPUT_DIR', GENAI_ROOT / 'outputs'))
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # Rapport JSON
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        report_file = output_dir / f'environment_validation_{timestamp}.json'
        
        with open(report_file, 'w', encoding='utf-8') as f:
            json.dump(validation_results, f, indent=2, ensure_ascii=False, default=str)
        
        print(f"💾 Rapport sauvegardé: {report_file}")
        
        # Export configuration validée
        if export_config and validation_results['overall_status'] == 'success':
            config_export = {
                'genai_environment': {
                    'version': '2.1.0',
                    'validated_at': validation_results['timestamp'],
                    'status': 'ready',
                    'apis': {
                        'openrouter': validation_results['tests'].get('apis', {}).get('details', {}).get('openrouter', {}),
                        'openai': validation_results['tests'].get('apis', {}).get('details', {}).get('openai', {})
                    },
                    'docker_services': validation_results['tests'].get('docker', {}).get('details', {}),
                    'notebooks_count': validation_results['tests'].get('notebooks', {}).get('total_count', 0)
                }
            }
            
            config_file = output_dir / f'genai_config_validated_{timestamp}.json'
            with open(config_file, 'w', encoding='utf-8') as f:
                json.dump(config_export, f, indent=2, ensure_ascii=False, default=str)
            
            print(f"⚙️  Configuration exportée: {config_file}")
        
    except Exception as e:
        print(f"❌ Erreur sauvegarde: {str(e)[:100]}...")

# Export final pour autres notebooks
ENVIRONMENT_VALIDATION_RESULT = validation_results

print(f"\n📊 Variables exportées: ENVIRONMENT_VALIDATION_RESULT")
print(f"🏁 Environment Validation complété - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"➡️  Module Environment terminé - Passer aux notebooks Images Foundation")